# Mov to Mp4 converter

In [ ]:
import os
from moviepy.editor import VideoFileClip

def convert_mov_to_mp4(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all .mov files in the input folder
    mov_files = [f for f in os.listdir(input_folder) if f.endswith('.mov')]

    for mov_file in mov_files:
        mov_path = os.path.join(input_folder, mov_file)
        mp4_file = os.path.splitext(mov_file)[0] + '.mp4'
        mp4_path = os.path.join(output_folder, mp4_file)

        # Load the .mov file
        video = VideoFileClip(mov_path)

        # Convert and save as .mp4
        video.write_videofile(mp4_path, codec='libx264', audio_codec="aac", temp_audiofile='temp.m4a', remove_temp=True)

        # Close the video file
        video.close()


# Specify the input and output folder paths
input_folder = "Replay Attack dataset\\test\\real"
output_folder = "replay\\test\\real"

# Call the function to convert .mov files to .mp4
convert_mov_to_mp4(input_folder, output_folder)

# Frame extraction
Works for mp4, avi, mkv

In [ ]:
import os
import cv2

def extract_frames(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all video files in the input folder
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mkv'))]
    print(video_files)

    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        
        # Open the video file
        video_capture = cv2.VideoCapture(video_path)

        # Get the total number of frames in the video
        total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate the frame interval to extract 5 frames
        frame_interval = max(total_frames // 5, 1)

        # Extract frames
        frame_count = 0
        frame_number = 0

        while frame_count < 5:
            # Read the next frame
            success, frame = video_capture.read()

            if not success:
                break

            # Save the frame to the output folder
            frame_output_path = os.path.join(output_folder, f"{video_file}_frame{frame_number}.jpg")
            cv2.imwrite(frame_output_path, frame)

            frame_count += 1
            frame_number += frame_interval

        # Release the video capture object
        video_capture.release()


# Specify the input and output folder paths
input_folder = "MSU dataset\\train\\real"
output_folder = "data/msu/train/real"

# Call the function to extract frames from videos
extract_frames(input_folder, output_folder)

# Face cropper
Testing Viola Jones using the webcam

In [ ]:
import cv2
import os
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface)

video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
        faceROI = frame[y:y+h,x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)

        # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

Cropping the dataset

In [ ]:
import cv2
import os
from tqdm import tqdm

def crop_face_from_image(image_path):
    # Load Haar cascades for face detection
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    faceCascade = cv2.CascadeClassifier(cascPathface)

    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

    # Check if any face is detected
    if len(faces) > 0:
        # Assuming the first detected face is the primary one
        x, y, w, h = faces[0]

        # Crop the face and make it square
        face_length = max(w, h)
        x_center, y_center = x + w // 2, y + h // 2
        x_new = max(x_center - face_length // 2, 0)
        y_new = max(y_center - face_length // 2, 0)
        cropped_face = image[y_new:y_new + face_length, x_new:x_new + face_length]

        # Save the cropped face, overwriting the original image
        cv2.imwrite(image_path, cropped_face)

def process_images_in_folder(input_folder):
    # Process each file in the input folder
    for filename in tqdm(os.listdir(input_folder)):
        file_path = os.path.join(input_folder, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            crop_face_from_image(file_path)
            #print(f"Processed and overwritten {filename}")

# Example usage
process_images_in_folder('data_crop/replay/train/real')

Delete images without faces using another face detection technique

In [ ]:
import cv2
import os
from mtcnn import MTCNN

def is_face_present(image_path, detector):
    image = cv2.imread(image_path)
    result = detector.detect_faces(image)
    return len(result) > 0

def delete_images_without_faces(root_folder, detector):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)
                if not is_face_present(file_path, detector):
                    os.remove(file_path)
                    print(f"Deleted {file_path}")

# Initialize MTCNN detector
detector = MTCNN()

# Process the "data_crop" folder
delete_images_without_faces('data_crop', detector)

Make sure that all the images are square

In [5]:
import cv2
import os

def crop_center_square(image_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    cropped_image = image[top:top + min_side, left:left + min_side]
    cv2.imwrite(image_path, cropped_image)

def process_images_in_folder(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                crop_center_square(image_path)
                print(f"Cropped {image_path}")

process_images_in_folder('data_crop')

Cropped data_crop\casia\test\attack\3 (10).avi_frame0.jpg
Cropped data_crop\casia\test\attack\3 (10).avi_frame105.jpg
Cropped data_crop\casia\test\attack\3 (10).avi_frame140.jpg
Cropped data_crop\casia\test\attack\3 (10).avi_frame35.jpg
Cropped data_crop\casia\test\attack\3 (10).avi_frame70.jpg
Cropped data_crop\casia\test\attack\3 (11).avi_frame0.jpg
Cropped data_crop\casia\test\attack\3 (11).avi_frame126.jpg
Cropped data_crop\casia\test\attack\3 (11).avi_frame168.jpg
Cropped data_crop\casia\test\attack\3 (11).avi_frame42.jpg
Cropped data_crop\casia\test\attack\3 (11).avi_frame84.jpg
Cropped data_crop\casia\test\attack\3 (12).avi_frame0.jpg
Cropped data_crop\casia\test\attack\3 (12).avi_frame116.jpg
Cropped data_crop\casia\test\attack\3 (12).avi_frame29.jpg
Cropped data_crop\casia\test\attack\3 (12).avi_frame58.jpg
Cropped data_crop\casia\test\attack\3 (12).avi_frame87.jpg
Cropped data_crop\casia\test\attack\3 (13).avi_frame0.jpg
Cropped data_crop\casia\test\attack\3 (13).avi_frame112

Resize the image to 224x224

In [1]:
import cv2
import os
from tqdm import tqdm

def resize_image(image_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    # Resize only if the image is larger than 224x224
    if h > 224 or w > 224:
        # Resize while maintaining aspect ratio
        dim = (224, int(224 * (h / w))) if w > h else (int(224 * (w / h)), 224)
        resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        cv2.imwrite(image_path, resized_image)
        print(f"Resized {image_path}")

def process_images_in_folder(root_folder):
    for root, dirs, files in os.walk(root_folder):
        # Wrap the file processing with tqdm for a progress bar
        for file in tqdm(files, desc=f"Processing in {root}"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                resize_image(image_path)

process_images_in_folder('data_crop')

Processing in data_crop: 0it [00:00, ?it/s]
Processing in data_crop\casia: 0it [00:00, ?it/s]
Processing in data_crop\casia\test: 0it [00:00, ?it/s]
Processing in data_crop\casia\test\attack:  11%|█         | 92/850 [00:00<00:01, 422.04it/s]

Resized data_crop\casia\test\attack\3 (10).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (10).avi_frame105.jpg
Resized data_crop\casia\test\attack\3 (10).avi_frame140.jpg
Resized data_crop\casia\test\attack\3 (10).avi_frame35.jpg
Resized data_crop\casia\test\attack\3 (10).avi_frame70.jpg
Resized data_crop\casia\test\attack\3 (12).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (12).avi_frame116.jpg
Resized data_crop\casia\test\attack\3 (12).avi_frame29.jpg
Resized data_crop\casia\test\attack\3 (12).avi_frame58.jpg
Resized data_crop\casia\test\attack\3 (12).avi_frame87.jpg
Resized data_crop\casia\test\attack\3 (13).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (13).avi_frame112.jpg
Resized data_crop\casia\test\attack\3 (13).avi_frame28.jpg
Resized data_crop\casia\test\attack\3 (13).avi_frame56.jpg
Resized data_crop\casia\test\attack\3 (13).avi_frame84.jpg
Resized data_crop\casia\test\attack\3 (18).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (18).avi_frame110.

Processing in data_crop\casia\test\attack:  26%|██▌       | 223/850 [00:00<00:01, 582.72it/s]

Resized data_crop\casia\test\attack\3 (6).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (6).avi_frame120.jpg
Resized data_crop\casia\test\attack\3 (6).avi_frame160.jpg
Resized data_crop\casia\test\attack\3 (6).avi_frame40.jpg
Resized data_crop\casia\test\attack\3 (6).avi_frame80.jpg
Resized data_crop\casia\test\attack\3 (7).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (7).avi_frame111.jpg
Resized data_crop\casia\test\attack\3 (7).avi_frame148.jpg
Resized data_crop\casia\test\attack\3 (7).avi_frame37.jpg
Resized data_crop\casia\test\attack\3 (7).avi_frame74.jpg
Resized data_crop\casia\test\attack\3 (8).avi_frame0.jpg
Resized data_crop\casia\test\attack\3 (8).avi_frame117.jpg
Resized data_crop\casia\test\attack\3 (8).avi_frame156.jpg
Resized data_crop\casia\test\attack\3 (8).avi_frame39.jpg
Resized data_crop\casia\test\attack\3 (8).avi_frame78.jpg
Resized data_crop\casia\test\attack\3.avi_frame0.jpg
Resized data_crop\casia\test\attack\3.avi_frame126.jpg
Resized data_crop\c

Processing in data_crop\casia\test\attack:  34%|███▍      | 287/850 [00:00<00:00, 598.12it/s]

...


Processing in data_crop\replay\train: 0it [00:00, ?it/s]
Processing in data_crop\replay\train\attack:  36%|███▌      | 534/1498 [00:00<00:00, 1292.13it/s]

Resized data_crop\replay\train\attack\attack_highdef_client027_session01_highdef_photo_adverse (2).mp4_frame0.jpg
Resized data_crop\replay\train\attack\attack_highdef_client027_session01_highdef_photo_adverse (2).mp4_frame138.jpg
Resized data_crop\replay\train\attack\attack_highdef_client027_session01_highdef_photo_adverse (2).mp4_frame184.jpg
Resized data_crop\replay\train\attack\attack_highdef_client027_session01_highdef_photo_adverse (2).mp4_frame92.jpg
Resized data_crop\replay\train\attack\attack_highdef_client027_session01_highdef_video_adverse (2).mp4_frame184.jpg


Processing in data_crop\replay\train\attack:  83%|████████▎ | 1236/1498 [00:00<00:00, 1365.88it/s]

Resized data_crop\replay\train\attack\attack_mobile_client027_session01_mobile_photo_adverse (2).mp4_frame0.jpg
Resized data_crop\replay\train\attack\attack_mobile_client027_session01_mobile_photo_adverse (2).mp4_frame138.jpg
Resized data_crop\replay\train\attack\attack_mobile_client027_session01_mobile_photo_adverse (2).mp4_frame184.jpg
Resized data_crop\replay\train\attack\attack_mobile_client027_session01_mobile_photo_adverse (2).mp4_frame46.jpg
Resized data_crop\replay\train\attack\attack_mobile_client027_session01_mobile_photo_adverse (2).mp4_frame92.jpg


Processing in data_crop\replay\train\real: 100%|██████████| 300/300 [00:00<00:00, 1638.14it/s]
